# Ensemble (CNN+GRU) Classification with CR Dataset
<hr>

We will build a text classification model using CNN and GRU models ensembled on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


sentence  label  split
0     weaknesses are minor the feel and layout of th...      0  train
1     many of our disney movies do n 't play on this...      0  train
2     player has a problem with dual layer dvd 's su...      0  train
3     i know the saying is you get what you pay for ...      0  train
4                      will never purchase apex again .      0  train
...                                                 ...    ...    ...
3770  so far , the anti spam feature seems to be ver...      1  train
3771  i downloaded a trial version of computer assoc...      1  train
3772  i did not have any of the installation problem...      1  train
3773  their products have been great and have saved ...      1  train
3774                                                         1  train

[3775 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          1368   1368
1          2407   2407

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  will never purchase apex again .
Into a sequence of int: [72, 194, 285, 207, 286]
Into a padded sequence: [ 72 194 285 207 286   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
and 3
i 4
it 5
to 6
a 7
is 8
of 9
this 10
5336


# Model 1: Embedding Random
<hr>

A __standard model__ for document classification is to use (quoted from __Jason Brownlee__, the author of [machinelearningmastery.com](https://machinelearningmastery.com)):
>- Word Embedding: A distributed representation of words where different words that have a similar meaning (based on their usage) also have a similar representation.
>- Convolutional Model: A feature extraction model that learns to extract salient features from documents represented using a word embedding.
>- Fully Connected Model: The interpretation of extracted features in terms of a predictive output.


Therefore, the model is comprised of the following elements:
- __Input layer__ that defines the length of input sequences.
- __Embedding layer__ set to the size of the vocabulary and 100-dimensional real-valued representations.
- __Conv1D layer__ with 32 filters and a kernel size set to the number of words to read at once.
- __MaxPooling1D layer__ to consolidate the output from the convolutional layer.
- __Flatten layer__ to reduce the three-dimensional output to two dimensional for concatenation.

The CNN model is inspired by __Yoon Kim__ paper in his study on the use of Word Embedding + CNN for text classification. The hyperparameters we use based on his study are as follows:
- Transfer function: rectified linear.
- Kernel sizes: 3, 4, 5.
- Number of filters: 100.
- Dropout rate: 0.5.
- Weight regularization (L2): 3.
- Batch Size: 50.
- Update Rule: Adam

We will perform the best parameter using __grid search__ and 10-fold cross validation.

## CNN-GRU Model

Now, we will build Convolutional Neural Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from `CNN for Sentence Classification` by *Yoon Kim*.

Now, we will define our CNN model as follows:
- One Conv layer with 100 filters, kernel size 5, and relu activation function;
- One MaxPool layer with pool size = 2;
- One Dropout layer after flattened;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dropout, MaxPool1D, Flatten, Dense, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate

In [12]:
def define_model(filters = 100, kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length, mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [13]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 98, 100)      90100       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 49, 100)      0           conv1d[0][0]                     
______________________________________________________________________________________________

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            # Define the input shape
            model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model(filters, kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                    
                else:
                    print('Done!')

            # evaluate the model
            loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
68/68 [==============================] - 59s 582ms/step - loss: 0.6973 - accuracy: 0.5173 - val_loss: 0.6530 - val_accuracy: 0.6349
Epoch 2/100
68/68 [==============================] - 25s 379ms/step - loss: 0.6424 - accuracy: 0.5922 - val_loss: 0.6186 - val_accuracy: 0.6349
Epoch 3/100
68/68 [==============================] - 31s 456ms/step - loss: 0.6048 - accuracy: 0.6577 - val_loss: 0.5978 - val_accuracy: 0.6349
Epoch 4/100
68/68 [==============================] - 26s 387ms/step - loss: 0.5714 - accuracy: 0.6762 - val_loss: 0.5902 - val_accuracy: 0.6349
Epoch 5/100
68/68 [==============================] - 26s 387ms/step - loss: 0.5456 - accuracy: 0.6763 - val_loss: 0.5804 - val_accuracy: 0.6349
Epoch 6/100
68/68 [==============================] - 26s 389ms/step - loss: 0.5258 - accuracy: 0.6856 - val_loss: 0.5750 - val_accuracy: 0.6349
Epoch 7/100
68/68 [==============================] - 27s 397ms/step - loss: 0.5133 - accuracy: 0.6802 - val_loss: 0.5681 - val_accuracy:

68/68 [==============================] - 27s 394ms/step - loss: 0.3313 - accuracy: 0.9636 - val_loss: 0.4881 - val_accuracy: 0.7798
Epoch 8/100
68/68 [==============================] - 27s 395ms/step - loss: 0.3117 - accuracy: 0.9653 - val_loss: 0.4832 - val_accuracy: 0.7878
Epoch 9/100
68/68 [==============================] - 27s 409ms/step - loss: 0.2979 - accuracy: 0.9696 - val_loss: 0.4797 - val_accuracy: 0.7825
Epoch 10/100
68/68 [==============================] - 26s 393ms/step - loss: 0.2850 - accuracy: 0.9737 - val_loss: 0.4794 - val_accuracy: 0.7878
Epoch 11/100
68/68 [==============================] - 27s 402ms/step - loss: 0.2696 - accuracy: 0.9748 - val_loss: 0.4886 - val_accuracy: 0.7613
Epoch 12/100
68/68 [==============================] - 27s 395ms/step - loss: 0.2575 - accuracy: 0.9781 - val_loss: 0.4728 - val_accuracy: 0.7613
Epoch 13/100
68/68 [==============================] - 26s 393ms/step - loss: 0.2339 - accuracy: 0.9831 - val_loss: 0.5027 - val_accuracy: 0.7692


68/68 [==============================] - 31s 460ms/step - loss: 0.2411 - accuracy: 0.9836 - val_loss: 0.5334 - val_accuracy: 0.7566
Epoch 14/100
68/68 [==============================] - 31s 455ms/step - loss: 0.2378 - accuracy: 0.9832 - val_loss: 0.5305 - val_accuracy: 0.7540
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 77.51322984695435
Done!
Test Accuracy: 77.51322984695435
Epoch 1/100
68/68 [==============================] - 49s 513ms/step - loss: 0.6682 - accuracy: 0.6122 - val_loss: 0.5806 - val_accuracy: 0.6587
Epoch 2/100
68/68 [==============================] - 30s 451ms/step - loss: 0.5632 - accuracy: 0.7153 - val_loss: 0.5756 - val_accuracy: 0.7751
Epoch 3/100
68/68 [==============================] - 30s 453ms/step - loss: 0.5420 - accuracy: 0.8350 - val_loss: 0.5589 - val_accuracy: 0.7619
Epoch 4/100
68/68 [==============================] - 30s 447ms/step - loss: 0.4836 - accuracy: 0.8917 - val_loss: 0.5072 - val_accuracy

68/68 [==============================] - 30s 442ms/step - loss: 0.3974 - accuracy: 0.9643 - val_loss: 0.5665 - val_accuracy: 0.7533
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 77.45358347892761
Done!
Test Accuracy: 77.45358347892761
Epoch 1/100
68/68 [==============================] - 44s 453ms/step - loss: 0.6516 - accuracy: 0.6330 - val_loss: 0.5355 - val_accuracy: 0.7825
Epoch 2/100
68/68 [==============================] - 26s 387ms/step - loss: 0.5107 - accuracy: 0.8301 - val_loss: 0.5174 - val_accuracy: 0.7798
Epoch 3/100
68/68 [==============================] - 26s 387ms/step - loss: 0.4444 - accuracy: 0.8936 - val_loss: 0.4921 - val_accuracy: 0.7984
Epoch 4/100
68/68 [==============================] - 26s 383ms/step - loss: 0.3813 - accuracy: 0.9428 - val_loss: 0.4864 - val_accuracy: 0.8037
Epoch 5/100
68/68 [==============================] - 26s 383ms/step - loss: 0.3519 - accuracy: 0.9587 - val_loss: 0.4657 - val_accuracy:

...

68/68 [==============================] - 30s 453ms/step - loss: 0.3636 - accuracy: 0.9566 - val_loss: 0.5347 - val_accuracy: 0.7672
Epoch 10/100
68/68 [==============================] - 30s 447ms/step - loss: 0.3617 - accuracy: 0.9425 - val_loss: 0.5145 - val_accuracy: 0.7804
Epoch 11/100
68/68 [==============================] - 30s 448ms/step - loss: 0.3496 - accuracy: 0.9601 - val_loss: 0.5190 - val_accuracy: 0.7725
Epoch 12/100
68/68 [==============================] - 31s 463ms/step - loss: 0.3353 - accuracy: 0.9507 - val_loss: 0.5304 - val_accuracy: 0.7566
Epoch 13/100
68/68 [==============================] - 30s 446ms/step - loss: 0.3213 - accuracy: 0.9559 - val_loss: 0.5298 - val_accuracy: 0.7646
Epoch 14/100
68/68 [==============================] - 30s 449ms/step - loss: 0.3197 - accuracy: 0.9748 - val_loss: 0.5172 - val_accuracy: 0.7831
Epoch 15/100
68/68 [==============================] - 31s 455ms/step - loss: 0.3111 - accuracy: 0.9753 - val_loss: 0.5099 - val_accuracy: 0.772

68/68 [==============================] - 27s 397ms/step - loss: 0.3054 - accuracy: 0.9789 - val_loss: 0.5352 - val_accuracy: 0.7772
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Test Accuracy: 80.63660264015198
Done!
Test Accuracy: 80.63660264015198
Epoch 1/100
68/68 [==============================] - 45s 461ms/step - loss: 0.7139 - accuracy: 0.4405 - val_loss: 0.6860 - val_accuracy: 0.6207
Epoch 2/100
68/68 [==============================] - 26s 389ms/step - loss: 0.6840 - accuracy: 0.6265 - val_loss: 0.6714 - val_accuracy: 0.6897
Epoch 3/100
68/68 [==============================] - 22s 333ms/step - loss: 0.6545 - accuracy: 0.7755 - val_loss: 0.6500 - val_accuracy: 0.7321
Epoch 4/100
68/68 [==============================] - 27s 397ms/step - loss: 0.6215 - accuracy: 0.8560 - val_loss: 0.6345 - val_accuracy: 0.7507
Epoch 5/100
68/68 [==============================] - 26s 391ms/step - loss: 0.5907 - accuracy: 0.9040 - val_loss: 0.6179 - val_accuracy:

68/68 [==============================] - 31s 462ms/step - loss: 0.6320 - accuracy: 0.6383 - val_loss: 0.6321 - val_accuracy: 0.5926
Epoch 3/100
68/68 [==============================] - 31s 460ms/step - loss: 0.5900 - accuracy: 0.6500 - val_loss: 0.6205 - val_accuracy: 0.5926
Epoch 4/100
68/68 [==============================] - 26s 391ms/step - loss: 0.5747 - accuracy: 0.6337 - val_loss: 0.6230 - val_accuracy: 0.5926
Epoch 5/100
68/68 [==============================] - 31s 458ms/step - loss: 0.5492 - accuracy: 0.6409 - val_loss: 0.6128 - val_accuracy: 0.5926
Epoch 6/100
68/68 [==============================] - 31s 459ms/step - loss: 0.5313 - accuracy: 0.6473 - val_loss: 0.6081 - val_accuracy: 0.5926
Epoch 7/100
68/68 [==============================] - 31s 459ms/step - loss: 0.5178 - accuracy: 0.6495 - val_loss: 0.6009 - val_accuracy: 0.5926
Epoch 8/100
68/68 [==============================] - 31s 455ms/step - loss: 0.5066 - accuracy: 0.6478 - val_loss: 0.6055 - val_accuracy: 0.5926
Rest

68/68 [==============================] - 29s 435ms/step - loss: 0.2930 - accuracy: 0.9763 - val_loss: 0.4812 - val_accuracy: 0.7905
Epoch 12/100
68/68 [==============================] - 25s 369ms/step - loss: 0.2694 - accuracy: 0.9751 - val_loss: 0.4821 - val_accuracy: 0.7958
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 81.69761300086975
Done!
Test Accuracy: 81.69761300086975

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  78.042328  77.248675  82.804233  79.629630  83.862436   
1       relu       2  78.306878  77.513230  81.481481  79.100531  82.804233   
2       relu       3  81.481481  79.365081  77.513230  81.216931  82.275134   
3       relu       4  78.306878  82.010579  77.777779  80.158728  84.126985   
4       relu       5  80.952382  77.777779  83.597887  76.455027  79.365081   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  80.901855  79.0450

68/68 [==============================] - 43s 441ms/step - loss: 0.6596 - accuracy: 0.6078 - val_loss: 0.5795 - val_accuracy: 0.6950
Epoch 2/100
68/68 [==============================] - 25s 376ms/step - loss: 0.5388 - accuracy: 0.7968 - val_loss: 0.5614 - val_accuracy: 0.7586
Epoch 3/100
68/68 [==============================] - 25s 376ms/step - loss: 0.4984 - accuracy: 0.8862 - val_loss: 0.5639 - val_accuracy: 0.7480
Epoch 4/100
68/68 [==============================] - 25s 376ms/step - loss: 0.4752 - accuracy: 0.9100 - val_loss: 0.5435 - val_accuracy: 0.7851
Epoch 5/100
68/68 [==============================] - 26s 379ms/step - loss: 0.4461 - accuracy: 0.9451 - val_loss: 0.5378 - val_accuracy: 0.7719
Epoch 6/100
68/68 [==============================] - 25s 375ms/step - loss: 0.4248 - accuracy: 0.9566 - val_loss: 0.5323 - val_accuracy: 0.7825
Epoch 7/100
68/68 [==============================] - 25s 377ms/step - loss: 0.4088 - accuracy: 0.9615 - val_loss: 0.5224 - val_accuracy: 0.7825
Epoc

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
5       relu       6  82.275134  82.804233  82.539684  80.158728  79.365081   
2       relu       3  81.481481  79.365081  77.513230  81.216931  82.275134   
3       relu       4  78.306878  82.010579  77.777779  80.158728  84.126985   
0       relu       1  78.042328  77.248675  82.804233  79.629630  83.862436   
4       relu       5  80.952382  77.777779  83.597887  76.455027  79.365081   
1       relu       2  78.306878  77.513230  81.481481  79.100531  82.804233   

        acc6       acc7       acc8       acc9      acc10        AVG  
5  78.779840  83.289123  78.779840  82.228118  78.514588  80.873437  
2  81.432360  79.840851  81.167108  81.697613  79.045093  80.503488  
3  83.023870  80.636603  78.249335  79.045093  81.432360  80.476821  
0  80.901855  79.045093  81.697613  79.840851  79.575598  80.264831  
4  76.392573  81.962866  81.962866  80.106103  81.697613  80.027018  
1  79.310346  78.779840  80.901855  77.453583  81.432360  79.708434

In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        80.873437

In [18]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_CR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [24]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [25]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## MLP-CNN-GRU Model

In [26]:
def ensemble_CNN_BiGRU(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel BiGRU
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [27]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_70"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_141 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_140 (Embedding)       (None, 100, 300)     300000      input_141[0][0]                  
__________________________________________________________________________________________________
conv1d_70 (Conv1D)              (None, 98, 100)      90100       embedding_140[0][0]              
__________________________________________________________________________________________________
max_pooling1d_70 (MaxPooling1D) (None, 49, 100)      0           conv1d_70[0][0]                  
___________________________________________________________________________________________

## Train and Test the Model

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=7, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
68/68 [==============================] - 43s 413ms/step - loss: 0.6541 - accuracy: 0.6251 - val_loss: 0.5755 - val_accuracy: 0.6561
Epoch 2/100
68/68 [==============================] - 22s 333ms/step - loss: 0.5680 - accuracy: 0.6512 - val_loss: 0.5680 - val_accuracy: 0.6561
Epoch 3/100
68/68 [==============================] - 23s 346ms/step - loss: 0.5459 - accuracy: 0.6396 - val_loss: 0.5291 - val_accuracy: 0.6561
Epoch 4/100
68/68 [==============================] - 24s 354ms/step - loss: 0.5359 - accuracy: 0.6617 - val_loss: 0.5156 - val_accuracy: 0.7116
Epoch 5/100
68/68 [==============================] - 25s 372ms/step - loss: 0.5195 - accuracy: 0.7725 - val_loss: 0.5050 - val_accuracy: 0.7989
Epoch 6/100
68/68 [==============================] - 23s 344ms/step - loss: 0.5031 - accuracy: 0.8075 - val_loss: 0.5121 - val_accuracy: 0.8042
Epoch 7/100
68/68 [==============================] - 23s 337ms/step - loss: 0.4883 - accuracy: 0.8399 - val_loss: 0.4994 - val_accuracy:

68/68 [==============================] - 21s 315ms/step - loss: 0.6682 - accuracy: 0.6404 - val_loss: 0.6696 - val_accuracy: 0.6243
Epoch 6/100
68/68 [==============================] - 25s 374ms/step - loss: 0.6637 - accuracy: 0.6441 - val_loss: 0.6672 - val_accuracy: 0.6243
Epoch 7/100
68/68 [==============================] - 25s 374ms/step - loss: 0.6606 - accuracy: 0.6449 - val_loss: 0.6657 - val_accuracy: 0.6243
Epoch 8/100
68/68 [==============================] - 25s 315ms/step - loss: 0.6614 - accuracy: 0.6364 - val_loss: 0.6644 - val_accuracy: 0.6243
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 62.43386268615723
The model suffered from local minimum. Retrain the model!
Epoch 1/100
68/68 [==============================] - 37s 371ms/step - loss: 0.6598 - accuracy: 0.6371 - val_loss: 0.6270 - val_accuracy: 0.6243
Epoch 2/100
68/68 [==============================] - 25s 372ms/step - loss: 0.6106 - accuracy: 0.6361 - val_loss: 0.6

68/68 [==============================] - 29s 429ms/step - loss: 0.2722 - accuracy: 0.9268 - val_loss: 0.4568 - val_accuracy: 0.8037
Epoch 25/100
68/68 [==============================] - 23s 266ms/step - loss: 0.2481 - accuracy: 0.9470 - val_loss: 0.4484 - val_accuracy: 0.7931
Epoch 26/100
68/68 [==============================] - 29s 429ms/step - loss: 0.2527 - accuracy: 0.9407 - val_loss: 0.4501 - val_accuracy: 0.8037
Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping
Test Accuracy: 80.90185523033142
Done!
Epoch 1/100
68/68 [==============================] - 78s 951ms/step - loss: 0.7197 - accuracy: 0.5185 - val_loss: 0.6856 - val_accuracy: 0.6366
Epoch 2/100
68/68 [==============================] - 24s 364ms/step - loss: 0.6917 - accuracy: 0.6157 - val_loss: 0.6793 - val_accuracy: 0.6366
Epoch 3/100
68/68 [==============================] - 27s 397ms/step - loss: 0.6816 - accuracy: 0.6238 - val_loss: 0.6740 - val_accuracy: 0.6366
Epoch 4/100
68/68 [====

Epoch 17/100
68/68 [==============================] - 25s 368ms/step - loss: 0.3473 - accuracy: 0.9413 - val_loss: 0.4738 - val_accuracy: 0.8122
Epoch 18/100
68/68 [==============================] - 24s 357ms/step - loss: 0.3379 - accuracy: 0.9443 - val_loss: 0.4519 - val_accuracy: 0.8254
Epoch 19/100
68/68 [==============================] - 23s 348ms/step - loss: 0.3209 - accuracy: 0.9522 - val_loss: 0.4664 - val_accuracy: 0.8069
Epoch 20/100
68/68 [==============================] - 21s 318ms/step - loss: 0.3195 - accuracy: 0.9467 - val_loss: 0.4665 - val_accuracy: 0.8069
Epoch 21/100
68/68 [==============================] - 23s 347ms/step - loss: 0.3160 - accuracy: 0.9469 - val_loss: 0.4687 - val_accuracy: 0.8069
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Test Accuracy: 83.06878209114075
Done!
Epoch 1/100
68/68 [==============================] - 45s 413ms/step - loss: 0.6504 - accuracy: 0.6362 - val_loss: 0.5989 - val_accuracy: 0.6402
Epoch 2/

...

Epoch 12/100
68/68 [==============================] - 22s 325ms/step - loss: 0.4579 - accuracy: 0.8813 - val_loss: 0.4895 - val_accuracy: 0.8175
Epoch 13/100
68/68 [==============================] - 22s 326ms/step - loss: 0.4405 - accuracy: 0.8934 - val_loss: 0.4898 - val_accuracy: 0.8201
Epoch 14/100
68/68 [==============================] - 22s 324ms/step - loss: 0.4319 - accuracy: 0.9016 - val_loss: 0.4767 - val_accuracy: 0.8333
Epoch 15/100
68/68 [==============================] - 22s 323ms/step - loss: 0.4182 - accuracy: 0.9077 - val_loss: 0.4734 - val_accuracy: 0.8307
Epoch 16/100
68/68 [==============================] - 18s 263ms/step - loss: 0.4123 - accuracy: 0.9085 - val_loss: 0.4707 - val_accuracy: 0.8307
Epoch 17/100
68/68 [==============================] - 22s 324ms/step - loss: 0.3955 - accuracy: 0.9170 - val_loss: 0.4710 - val_accuracy: 0.8254
Epoch 18/100
68/68 [==============================] - 22s 325ms/step - loss: 0.4149 - accuracy: 0.8901 - val_loss: 0.4618 - val_ac

Epoch 17/100
68/68 [==============================] - 26s 381ms/step - loss: 0.3433 - accuracy: 0.9321 - val_loss: 0.5013 - val_accuracy: 0.7851
Epoch 18/100
68/68 [==============================] - 26s 384ms/step - loss: 0.3263 - accuracy: 0.9456 - val_loss: 0.5000 - val_accuracy: 0.7878
Epoch 19/100
68/68 [==============================] - 26s 321ms/step - loss: 0.3171 - accuracy: 0.9479 - val_loss: 0.5009 - val_accuracy: 0.7931
Epoch 20/100
68/68 [==============================] - 26s 382ms/step - loss: 0.3161 - accuracy: 0.9466 - val_loss: 0.4957 - val_accuracy: 0.7878
Epoch 21/100
68/68 [==============================] - 26s 384ms/step - loss: 0.2986 - accuracy: 0.9522 - val_loss: 0.4982 - val_accuracy: 0.7931
Epoch 22/100
68/68 [==============================] - 26s 382ms/step - loss: 0.3026 - accuracy: 0.9455 - val_loss: 0.4855 - val_accuracy: 0.7878
Epoch 23/100
68/68 [==============================] - 26s 383ms/step - loss: 0.2975 - accuracy: 0.9495 - val_loss: 0.4996 - val_ac

Epoch 4/100
68/68 [==============================] - 26s 380ms/step - loss: 0.6692 - accuracy: 0.5729 - val_loss: 0.6567 - val_accuracy: 0.6366
Epoch 5/100
68/68 [==============================] - 26s 380ms/step - loss: 0.6737 - accuracy: 0.5600 - val_loss: 0.6556 - val_accuracy: 0.6366
Epoch 6/100
68/68 [==============================] - 25s 379ms/step - loss: 0.6600 - accuracy: 0.6444 - val_loss: 0.6255 - val_accuracy: 0.6366
Epoch 7/100
68/68 [==============================] - 26s 381ms/step - loss: 0.6226 - accuracy: 0.6818 - val_loss: 0.6105 - val_accuracy: 0.6366
Epoch 8/100
68/68 [==============================] - 22s 320ms/step - loss: 0.6115 - accuracy: 0.6764 - val_loss: 0.6025 - val_accuracy: 0.6366
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 63.660478591918945
The model suffered from local minimum. Retrain the model!
Epoch 1/100
68/68 [==============================] - 42s 443ms/step - loss: 0.6954 - accuracy: 0.4731 - 

68/68 [==============================] - 32s 380ms/step - loss: 0.6640 - accuracy: 0.6304 - val_loss: 0.6609 - val_accuracy: 0.6349
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 63.49206566810608
The model suffered from local minimum. Retrain the model!
Epoch 1/100
68/68 [==============================] - 56s 584ms/step - loss: 0.7344 - accuracy: 0.4696 - val_loss: 0.6790 - val_accuracy: 0.6349
Epoch 2/100
68/68 [==============================] - 28s 420ms/step - loss: 0.6774 - accuracy: 0.6311 - val_loss: 0.6705 - val_accuracy: 0.6349
Epoch 3/100
68/68 [==============================] - 28s 413ms/step - loss: 0.6665 - accuracy: 0.6472 - val_loss: 0.6646 - val_accuracy: 0.6349
Epoch 4/100
68/68 [==============================] - 27s 408ms/step - loss: 0.6630 - accuracy: 0.6376 - val_loss: 0.6611 - val_accuracy: 0.6349
Epoch 5/100
68/68 [==============================] - 28s 409ms/step - loss: 0.6595 - accuracy: 0.6378 - val_loss: 0.6

KeyboardInterrupt: 

## Summary

In [30]:
record2.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
1       relu       2  83.068782  81.746030  84.126985  83.068782  83.333331   
2       relu       3  80.423278  81.481481  85.185188  82.010579  83.597887   
0       relu       1  82.275134  82.539684  80.423278  82.539684  84.920633   
3       relu       4  79.894179  82.010579  84.391534  81.216931  79.629630   
4       relu       5  80.952382  84.126985  81.216931  81.746030  85.185188   
5       relu       6  82.804233  81.216931  82.804233  81.481481  83.597887   

        acc6       acc7       acc8       acc9      acc10        AVG  
1  84.350133  81.697613  81.167108  78.249335  83.819628  82.462773  
2  80.901855  84.615386  81.432360  80.901855  82.228118  82.277799  
0  84.350133  82.228118  80.901855  80.371350  81.962866  82.251273  
3  83.289123  80.636603  84.615386  83.289123  82.228118  82.120121  
4  79.310346  83.554375  81.697613  83.819628  78.779840  82.038932  
5  81.697613  80.106103  80.106103  82.758623  83.023870  81.959708

In [31]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

AVG
Activation           
relu        82.462773

In [32]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_CR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## MLP-CNN-GRU Model

In [35]:
def define_model_3(filters = 100, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1
    input1 = Input(shape=(max_length,))
    embeddding1 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True)(input1)
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= MaxNorm( max_value=3, axis=[0,1]))(embeddding1)
    pool1 = MaxPool1D(pool_size=2, strides=2)(conv1)
    flat1 = Flatten()(pool1)
    drop1 = Dropout(0.5)(flat1)
    dense1 = Dense(10, activation='relu')(drop1)
    drop1 = Dropout(0.5)(dense1)
    out1 = Dense(1, activation='sigmoid')(drop1)
    
    # Channel 2
    input2 = Input(shape=(max_length,))
    embeddding2 = Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = True,
                            mask_zero=True)(input2)
    gru2 = Bidirectional(GRU(64))(embeddding2)
    drop2 = Dropout(0.5)(gru2)
    out2 = Dense(1, activation='sigmoid')(drop2)
    
    # Merge
    merged = concatenate([out1, out2])
    
    # Interpretation
    outputs = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[input1, input2], outputs=outputs)
    
    # Compile
    model.compile( loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [36]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_196"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_393 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_392 (Embedding)       (None, 100, 300)     300000      input_393[0][0]                  
__________________________________________________________________________________________________
conv1d_196 (Conv1D)             (None, 98, 100)      90100       embedding_392[0][0]              
__________________________________________________________________________________________________
max_pooling1d_196 (MaxPooling1D (None, 49, 100)      0           conv1d_196[0][0]                 
__________________________________________________________________________________________

## Train and Test the Model

In [37]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [38]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
filters = 100
kernel_sizes = [1, 2, 3, 4, 5, 6]
emb_mean = emb_mean
emb_std = emb_std

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.7):
                
                # Train the model
                model.fit(x=[Xtrain, Xtrain], y = train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=([Xtest, Xtest], test_y))

                # evaluate the model
                loss, acc = model.evaluate([Xtest, Xtest], test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.7):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(filters, kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                    
                else:
                    print('Done!')
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
68/68 [==============================] - 51s 503ms/step - loss: 0.6899 - accuracy: 0.5500 - val_loss: 0.6403 - val_accuracy: 0.6746
Epoch 2/100
68/68 [==============================] - 28s 410ms/step - loss: 0.6210 - accuracy: 0.7213 - val_loss: 0.5532 - val_accuracy: 0.8095
Epoch 3/100
68/68 [==============================] - 34s 501ms/step - loss: 0.5107 - accuracy: 0.8821 - val_loss: 0.5349 - val_accuracy: 0.8148
Epoch 4/100
68/68 [==============================] - 33s 485ms/step - loss: 0.4566 - accuracy: 0.9279 - val_loss: 0.5221 - val_accuracy: 0.8069
Epoch 5/100
68/68 [==============================] - 29s 425ms/step - loss: 0.4141 - accuracy: 0.9588 - val_loss: 0.5269 - val_accuracy: 0.7884
Epoch 6/100
68/68 [==============================] - 26s 391ms/step - loss: 0.3899 - accuracy: 0.9696 - val_loss: 0.5194 - val_accuracy: 0.7910
Epoch 7/100
68/68 [==============================] - 26s 393ms/step - loss: 0.3721 - accuracy: 0.9715 - val_loss: 0.5108 - val_accuracy:

Epoch 23/100
68/68 [==============================] - 29s 370ms/step - loss: 0.1664 - accuracy: 0.9912 - val_loss: 0.4905 - val_accuracy: 0.7910
Epoch 24/100
68/68 [==============================] - 53s 784ms/step - loss: 0.1546 - accuracy: 0.9919 - val_loss: 0.4868 - val_accuracy: 0.7910
Epoch 25/100
68/68 [==============================] - 25s 370ms/step - loss: 0.1594 - accuracy: 0.9864 - val_loss: 0.4874 - val_accuracy: 0.7910
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Test Accuracy: 79.89417910575867
Done!
Epoch 1/100
68/68 [==============================] - 46s 495ms/step - loss: 0.6613 - accuracy: 0.6258 - val_loss: 0.6144 - val_accuracy: 0.6825
Epoch 2/100
68/68 [==============================] - 31s 457ms/step - loss: 0.6315 - accuracy: 0.6313 - val_loss: 0.5894 - val_accuracy: 0.6825
Epoch 3/100
68/68 [==============================] - 29s 430ms/step - loss: 0.5868 - accuracy: 0.6282 - val_loss: 0.5305 - val_accuracy: 0.6825
Epoch 4/10

68/68 [==============================] - 25s 378ms/step - loss: 0.5826 - accuracy: 0.6535 - val_loss: 0.5939 - val_accuracy: 0.6101
Epoch 3/100
68/68 [==============================] - 25s 377ms/step - loss: 0.5562 - accuracy: 0.6468 - val_loss: 0.5863 - val_accuracy: 0.6101
Epoch 4/100
68/68 [==============================] - 21s 249ms/step - loss: 0.5259 - accuracy: 0.7267 - val_loss: 0.5758 - val_accuracy: 0.7639
Epoch 5/100
68/68 [==============================] - 25s 376ms/step - loss: 0.5059 - accuracy: 0.9063 - val_loss: 0.5682 - val_accuracy: 0.7533
Epoch 6/100
68/68 [==============================] - 21s 312ms/step - loss: 0.4757 - accuracy: 0.9422 - val_loss: 0.5522 - val_accuracy: 0.8011
Epoch 7/100
68/68 [==============================] - 25s 376ms/step - loss: 0.4617 - accuracy: 0.9551 - val_loss: 0.5527 - val_accuracy: 0.7825
Epoch 8/100
68/68 [==============================] - 25s 377ms/step - loss: 0.4449 - accuracy: 0.9631 - val_loss: 0.5402 - val_accuracy: 0.8064
Epoc

68/68 [==============================] - 26s 380ms/step - loss: 0.1957 - accuracy: 0.9738 - val_loss: 0.4706 - val_accuracy: 0.8042
Epoch 23/100
68/68 [==============================] - 26s 380ms/step - loss: 0.1847 - accuracy: 0.9816 - val_loss: 0.5239 - val_accuracy: 0.7513
Epoch 24/100
68/68 [==============================] - 25s 379ms/step - loss: 0.1757 - accuracy: 0.9756 - val_loss: 0.4589 - val_accuracy: 0.7937
Epoch 25/100
68/68 [==============================] - 26s 381ms/step - loss: 0.1707 - accuracy: 0.9802 - val_loss: 0.4740 - val_accuracy: 0.7963
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Test Accuracy: 80.95238208770752
Done!
Epoch 1/100
68/68 [==============================] - 52s 577ms/step - loss: 0.6369 - accuracy: 0.6512 - val_loss: 0.5883 - val_accuracy: 0.6138
Epoch 2/100
68/68 [==============================] - 26s 381ms/step - loss: 0.5460 - accuracy: 0.6936 - val_loss: 0.5472 - val_accuracy: 0.7381
Epoch 3/100
68/68 [===

...

68/68 [==============================] - 22s 322ms/step - loss: 0.4613 - accuracy: 0.9453 - val_loss: 0.5291 - val_accuracy: 0.8223
Epoch 14/100
68/68 [==============================] - 22s 324ms/step - loss: 0.4397 - accuracy: 0.9613 - val_loss: 0.5206 - val_accuracy: 0.8223
Epoch 15/100
68/68 [==============================] - 22s 325ms/step - loss: 0.4186 - accuracy: 0.9698 - val_loss: 0.5144 - val_accuracy: 0.8170
Epoch 16/100
68/68 [==============================] - 22s 324ms/step - loss: 0.4044 - accuracy: 0.9718 - val_loss: 0.5295 - val_accuracy: 0.7825
Epoch 17/100
68/68 [==============================] - 22s 325ms/step - loss: 0.3969 - accuracy: 0.9683 - val_loss: 0.5030 - val_accuracy: 0.8223
Epoch 18/100
68/68 [==============================] - 22s 331ms/step - loss: 0.3847 - accuracy: 0.9697 - val_loss: 0.4896 - val_accuracy: 0.8302
Epoch 19/100
68/68 [==============================] - 22s 325ms/step - loss: 0.3676 - accuracy: 0.9781 - val_loss: 0.4772 - val_accuracy: 0.838

Epoch 12/100
68/68 [==============================] - 30s 446ms/step - loss: 0.3672 - accuracy: 0.9787 - val_loss: 0.5244 - val_accuracy: 0.7910
Epoch 13/100
68/68 [==============================] - 30s 443ms/step - loss: 0.3560 - accuracy: 0.9761 - val_loss: 0.5198 - val_accuracy: 0.7989
Epoch 14/100
68/68 [==============================] - 30s 439ms/step - loss: 0.3411 - accuracy: 0.9793 - val_loss: 0.5207 - val_accuracy: 0.7910
Epoch 15/100
68/68 [==============================] - 30s 442ms/step - loss: 0.3234 - accuracy: 0.9849 - val_loss: 0.5097 - val_accuracy: 0.7857
Epoch 16/100
68/68 [==============================] - 30s 444ms/step - loss: 0.3202 - accuracy: 0.9781 - val_loss: 0.5373 - val_accuracy: 0.7804
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 80.68783283233643
Done!
Epoch 1/100
68/68 [==============================] - 44s 448ms/step - loss: 0.7415 - accuracy: 0.4626 - val_loss: 0.6877 - val_accuracy: 0.6164
Epoch 2/

68/68 [==============================] - 25s 371ms/step - loss: 0.4362 - accuracy: 0.9632 - val_loss: 0.5261 - val_accuracy: 0.8117
Epoch 10/100
68/68 [==============================] - 25s 371ms/step - loss: 0.4228 - accuracy: 0.9689 - val_loss: 0.5248 - val_accuracy: 0.7878
Epoch 11/100
68/68 [==============================] - 25s 370ms/step - loss: 0.4036 - accuracy: 0.9770 - val_loss: 0.5200 - val_accuracy: 0.8064
Epoch 12/100
68/68 [==============================] - 25s 368ms/step - loss: 0.3998 - accuracy: 0.9709 - val_loss: 0.5143 - val_accuracy: 0.7878
Epoch 13/100
68/68 [==============================] - 26s 387ms/step - loss: 0.3825 - accuracy: 0.9727 - val_loss: 0.5081 - val_accuracy: 0.7984
Epoch 14/100
68/68 [==============================] - 22s 331ms/step - loss: 0.3745 - accuracy: 0.9758 - val_loss: 0.5049 - val_accuracy: 0.7931
Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 83.02386999130249
Done!
Epoch 1/100
68/68 [=

Epoch 5/100
68/68 [==============================] - 29s 424ms/step - loss: 0.3851 - accuracy: 0.9310 - val_loss: 0.4812 - val_accuracy: 0.8042
Epoch 6/100
68/68 [==============================] - 29s 424ms/step - loss: 0.3427 - accuracy: 0.9538 - val_loss: 0.4665 - val_accuracy: 0.7963
Epoch 7/100
68/68 [==============================] - 29s 424ms/step - loss: 0.3177 - accuracy: 0.9601 - val_loss: 0.4488 - val_accuracy: 0.8307
Epoch 8/100
68/68 [==============================] - 29s 425ms/step - loss: 0.2961 - accuracy: 0.9635 - val_loss: 0.4411 - val_accuracy: 0.8148
Epoch 9/100
68/68 [==============================] - 29s 429ms/step - loss: 0.2726 - accuracy: 0.9737 - val_loss: 0.4260 - val_accuracy: 0.8333
Epoch 10/100
68/68 [==============================] - 29s 426ms/step - loss: 0.2632 - accuracy: 0.9754 - val_loss: 0.4314 - val_accuracy: 0.8228
Epoch 11/100
68/68 [==============================] - 29s 425ms/step - loss: 0.2465 - accuracy: 0.9754 - val_loss: 0.4281 - val_accurac

Epoch 5/100
68/68 [==============================] - 32s 475ms/step - loss: 0.5495 - accuracy: 0.6395 - val_loss: 0.5909 - val_accuracy: 0.6270
Epoch 6/100
68/68 [==============================] - 31s 462ms/step - loss: 0.5320 - accuracy: 0.6608 - val_loss: 0.5626 - val_accuracy: 0.7090
Epoch 7/100
68/68 [==============================] - 30s 449ms/step - loss: 0.4911 - accuracy: 0.7609 - val_loss: 0.5554 - val_accuracy: 0.7646
Epoch 8/100
68/68 [==============================] - 31s 464ms/step - loss: 0.4558 - accuracy: 0.8788 - val_loss: 0.5279 - val_accuracy: 0.8069
Epoch 9/100
68/68 [==============================] - 30s 444ms/step - loss: 0.4236 - accuracy: 0.9163 - val_loss: 0.5209 - val_accuracy: 0.7831
Epoch 10/100
68/68 [==============================] - 30s 444ms/step - loss: 0.4028 - accuracy: 0.9392 - val_loss: 0.5205 - val_accuracy: 0.7910
Epoch 11/100
68/68 [==============================] - 34s 441ms/step - loss: 0.3739 - accuracy: 0.9526 - val_loss: 0.5141 - val_accurac

## Summary

In [39]:
record3.sort_values(by='AVG', ascending=False)

Activation Filters       acc1       acc2       acc3       acc4       acc5  \
5       relu       6  85.449737  82.275134  84.126985  83.862436  80.687833   
0       relu       1  81.481481  83.597887  79.894179  85.185188  82.539684   
3       relu       4  82.010579  78.042328  84.920633  79.894179  83.862436   
4       relu       5  83.333331  80.687833  81.746030  80.952382  81.746030   
1       relu       2  80.952382  81.216931  82.804233  83.333331  80.952382   
2       relu       3  82.010579  79.629630  80.687833  82.275134  84.391534   

        acc6       acc7       acc8       acc9      acc10        AVG  
5  80.106103  79.575598  80.106103  83.819628  84.350133  82.435969  
0  83.023870  82.228118  79.045093  80.901855  80.106103  81.800346  
3  79.575598  82.758623  79.310346  81.962866  84.615386  81.695297  
4  83.819628  83.023870  80.636603  80.371350  80.371350  81.668841  
1  79.575598  79.045093  85.145891  79.575598  84.084880  81.668632  
2  77.453583  83.554375  82.228118  82.493371  80.901855  81.562601

In [40]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('HYBRID_CR_3.xlsx', sheet_name='dynamic')